# EDA Basic Lesson Plan

Welcome to the Exploratory Data Analysis (EDA) lesson! This notebook is designed to guide you through the essential steps of taking a raw dataset, understanding its structure, cleaning it, and transforming it for analysis.

**Structure:**
* **Section 1: The First Look:** How to read files and get a quick statistical summary.
* **Section 2: The Cleanup:** Handling the messy reality of data (missing values, duplicates, outliers).
* **Section 3: Refinement & Transformation:** preparing data for machine learning or deep analysis (mapping, string manipulation).

**For Instructors:** Use the narrative to introduce concepts before running code cells.
**For Learners:** Read the comments in the code to understand exactly what each parameter does.

### Setup
As always, we start by importing the necessary libraries. `pandas` is our primary tool for data manipulation, and `numpy` helps with numerical operations.

In [60]:
import pandas as pd
import numpy as np

pd.__version__

'2.1.4'

---

# Section 1: The First Look: Inspection & Summary

**Goal:** Load data from different file formats and get a "health check" of the data using descriptive statistics.

We will cover:
1. Reading CSV and Text files (handling headers, delimiters, and special characters).
2. Summarizing data using `.describe()`, `.sum()`, and `.value_counts()`.

## Part 1: Reading Data

Pandas is incredibly flexible with input formats. Let's start by reading a standard Comma-Separated Values (CSV) file.

In [2]:
# Let's inspect the raw file content first using a shell command
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
# Use read_csv to load the data into a DataFrame
# By default, it assumes the first row is the header
df = pd.read_csv("../data/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Scenario:** What if the file doesn't have a header row? If we don't specify this, pandas will mistakenly use the first row of data as column names.

In [4]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
# Option 1: Tell pandas there is no header. It will assign integers (0, 1, 2...) as column names.
pd.read_csv("../data/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# Option 2: Provide your own column names using the 'names' parameter
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Indexing:** You can also designate a specific column to be the index (row labels) of the DataFrame, rather than the default 0, 1, 2... index.

In [ ]:
names = ["a", "b", "c", "d", "message"]

# Use 'index_col' to set the 'message' column as the index
pd.read_csv("../data/ex2.csv", names=names, index_col="message")

# When to use this? when you have a natural identifier e.g. product ID, user ID etc.

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


**Irregular Separators:** Sometimes data isn't separated by commas. It might be tabs, spaces, or a variable amount of whitespace. We can use Regular Expressions (Regex) to handle this.

In [ ]:
# Inspect a file with messy whitespace
# this is not a CSV file, but a space-separated file
!cat ../data/ex3.txt

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [9]:
# sep="\s+" is a regex that means "one or more whitespace characters"
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


**Skipping Rows:** Sometimes files contain comments or metadata at the top that we want to ignore.

In [10]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [13]:
# Skip specific rows by index (0, 2, and 3 here) to get to the clean data
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Handling Missing Values (at load time):** Pandas is smart about identifying missing data (empty strings, 'NA', 'NULL'), but we can also define our own "sentinels" for missing values.

- Pandas auto-detects common missing value indicators like ' '. 'NA', 'NULL'.
- Sentinel - a special value in your data that should be treated as missing (e.g. -999 for temperature).

In [14]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,NULL

In [15]:
# Default behavior: reads 'NA' and 'NULL' as NaN (Not a Number/Missing)
result = pd.read_csv("../data/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [16]:
# Verify where the missing values are detected
result.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,True


In [ ]:
# these two instructions help to summarize the missing data info
result.isna().sum() # Count of missing values per column
result.isna().any() # Any missing values in each column

You can customize what is considered "Missing" for each column specifically.

In [19]:
# Define a dictionary of sentinels
# By default sentinel values are [' ', 'NA', 'NULL', '#N/A', 'nan'], but we can customize them
# For column 'message', treat 'foo' and 'NA' as null
# For column 'something', treat 'two' as null
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


**Reading Excel:** Reading Excel files works similarly, but we can specify sheet names.

In [20]:
# Load the Excel file object
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

# List available sheets
xlsx.sheet_names

['2017']

In [ ]:
# Parse a specific sheet into a DataFrame
xlsx.parse(sheet_name="2017")

In [ ]:
# Shortcut: Read directly without creating an ExcelFile object first
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

## Part 2: Summarizing and Computing Descriptive Statistics

Once data is loaded, we need to understand its "shape". We use summary statistics to get a birds-eye view.

> **📊 Visual Illustration Available**: See ![descriptive_statistics](../visualisations/01_descriptive_statistics.png) for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


Let's create a sample DataFrame with some missing values to test our summary methods.

In [122]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


**Reductions:** Methods like `.sum()` or `.mean()` reduce a Series of numbers to a single value. By default, they operate down the rows (`axis=0`).

In [125]:
# Sums down the rows (returns sum for each column)
df.sum() # axis=0 by default, equivalent to df.sum(axis='index')

one    9.25
two   -5.80
dtype: float64

In [126]:
# Sums across the columns (returns sum for each row)
# axis=1 is synonymous with axis='columns'
df.sum(axis="columns")  # df.sum(axis=1) 

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

**Handling NAs in Calculations:** By default, pandas ignores NaNs (treats them as zero for sums). We can change this with `skipna`.

In [127]:
# If any value is NaN, the result is NaN
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [28]:
df.sum(axis=1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

In [30]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


**Indirect Statistics:** Finding *where* the max or min value is located (the index label).

In [ ]:
# Returns the index of the maximum value
df.idxmax()

In [ ]:
# Returns the index of the minimum value
df.idxmin()

**Accumulations:** Computing cumulative sums.

In [ ]:
# Running total - useful for tracking accumulated values over time
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


**The `.describe()` method:** This is your best friend for a quick snapshot of numeric data.

In [ ]:
# Provides count, mean, std, min, quartiles, and max
# by default, df.describe() only analyzes numeric columns.  to include all columns use df.describe(include='all')
df.describe()

# count - non-missing values per column
# mean - average value per column
# std - standard deviation per column
# min - minimum value per column
# 25% - first quartile (25th percentile)
# 50% - second quartile (median)
# 75% - third quartile (75th percentile)
# max - maximum value per column

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


**Categorical/Non-Numeric Data:** `.describe()` behaves differently for string data, showing counts and uniqueness.

In [ ]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

# For object data, we get count, unique, top (most frequent), and freq
obj.describe()

# count: total items
# unique: number of unique items
# top: most frequent item
# freq: frequency of the most frequent item
# These are categorical summary

count     8
unique    4
top       c
freq      3
dtype: object

In [35]:
# Get unique values
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [36]:
# Get frequency counts of each unique value
obj.value_counts()

c    3
a    2
b    2
d    1
Name: count, dtype: int64

**Exercise:** Sort value counts in ascending order.

In [37]:
obj.value_counts().sort_index()

a    2
b    2
c    3
d    1
Name: count, dtype: int64

In [39]:
obj.value_counts(ascending=True)

d    1
a    2
b    2
c    3
Name: count, dtype: int64

---

# Section 2: The Cleanup: Missing Data, Duplicates & Outliers

**Goal:** Real-world data is dirty. In this section, we will learn techniques to identify and fix common quality issues.

We will cover:
1. **Missing Data:** Detecting nulls and deciding whether to drop them or fill them.
2. **Duplicates:** Finding and removing repeated records.
3. **Outliers:** identifying extreme values using boolean indexing.

## Part 1: Handling Missing Data

Missing data is often represented as `NaN` (Not a Number) or `None`.

> **📊 Visual Illustration Available**: See ![missing_data](../visualisations/03_missing_data.png) for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


In [40]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [41]:
# .isna() returns a boolean mask (True if missing, False if present)
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python `None` value is also treated as NA in pandas object arrays.

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"]) # None and np.nan are treated as NaN

string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [43]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

### Strategy 1: Dropping Missing Data (`dropna`)
The simplest strategy is to just remove the rows or columns that contain missing values.

In [45]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [ ]:
# Removes all NaN values from the Series
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [ ]:
# This is equivalent to boolean filtering:
data[data.notna()]

With DataFrames, `dropna` by default drops **any row** containing **any missing value**.

In [47]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [48]:
# Drops rows 1, 2, and 3 because they have at least one NaN
data.dropna()

,0,1,2
0,1.0,6.5,3.0


We can control this behavior. `how='all'` only drops rows where **all** values are NaN.

In [ ]:
data.dropna(how="all")

# keeps partial data because it only drops rows where all values are NaN

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop **columns** instead of rows, pass `axis=1`.

In [50]:
# Let's add a column of all NaNs first
data[4] = np.nan

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [51]:
# Drops the column '4' because it is all NaNs
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


We can also set a **threshold**: keep only rows containing at least `n` valid observations.

In [52]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# Set some missing values
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

,0,1,2
0,-2.029795,NaN,NaN
1,0.380852,NaN,NaN
2,0.706344,NaN,-0.216163
3,-0.865231,NaN,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


In [53]:
df.dropna()

,0,1,2
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


In [54]:
# Keep rows that have at least 2 non-NaN values
df.dropna(thresh=2)

,0,1,2
2,0.706344,NaN,-0.216163
3,-0.865231,NaN,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


### Strategy 2: Filling Missing Data (`fillna`)
Instead of losing data, we can fill the holes with a constant or a calculated value.

In [55]:
# Replace all NaNs with 0
df.fillna(0)

,0,1,2
0,-2.029795,0.000000,0.000000
1,0.380852,0.000000,0.000000
2,0.706344,0.000000,-0.216163
3,-0.865231,0.000000,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


You can specify different fill values for each column:

In [56]:
# Fill column 1 with 0.5, and column 2 with 0
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-2.029795,0.500000,0.000000
1,0.380852,0.500000,0.000000
2,0.706344,0.500000,-0.216163
3,-0.865231,0.500000,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


**Forward/Backward Fill:** Useful for time-series data, where you propagate the last valid observation forward or backward.

** Forward/Backward Fill:** Useful for time-series data, where you propagate the last valid observation forward or backward.

Time series data is a sequence of data points collected or recorded at specific time intervals. Each data point is associated with a timestamp, allowing for analysis of trends, patterns, and changes over time.
E.g. stock prices recorded every minute, daily temperature readings, or monthly sales figures, heart rates measurements over time

In time series data, the most recent value is often a best guess for a missing value.

This would be suitable for non time series data as well, especially when we believe that the missing value is similar to the previous or next value.-series data, especially when we believe that the missing value is similar to the previous or next value.

In [58]:
# Propagate next valid value backward to fill gaps
df.fillna(method="bfill")

/var/folders/vf/p6tql7n155x4cbnbq23vqytr0000gn/T/ipykernel_29753/3589963815.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill")


,0,1,2
0,-2.029795,-0.233582,-0.216163
1,0.380852,-0.233582,-0.216163
2,0.706344,-0.233582,-0.216163
3,-0.865231,-0.233582,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


In [ ]:
# Same, but limit how many rows are filled consecutively
df.fillna(method="bfill", limit=2)

# df.bfill(limit=2) --> same as above

/var/folders/vf/p6tql7n155x4cbnbq23vqytr0000gn/T/ipykernel_29753/12601219.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", limit=2)


,0,1,2
0,-2.029795,NaN,-0.216163
1,0.380852,NaN,-0.216163
2,0.706344,-0.233582,-0.216163
3,-0.865231,-0.233582,0.799387
4,0.518572,-0.233582,-1.650958
5,-0.179136,-0.961237,-0.463625
6,-0.247250,1.801889,2.180506


**Imputation:** Filling with the mean or median is a very common technique.

** Imputation:** Filling with the mean or median is a very common technique.

Imputation is the process of replacing missing data with substituted values. Filling missing values with the mean or median of the available data is a common imputation technique, especially for numerical datasets. This approach helps maintain the overall distribution of the data while minimizing bias introduced by missing values. 

In [61]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [62]:
# Fill with the mean of the available data
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Exercise: Handling Missing Data
Practice using these methods on the dataframe below.

In [63]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

,0,1,2
0,1.729562,1.432968,2.325303
1,-1.586890,0.291956,1.311468
2,-1.165746,NaN,-0.655313
3,NaN,-0.386667,-0.043223
4,NaN,NaN,NaN
5,-0.782933,NaN,NaN


> **Tasks:**
> 1. Remove rows with *any* missing values.
> 2. Remove rows with *all* missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

In [ ]:
# 1. Remove rows with *any* missing values.
df.dropna()

,0,1,2
0,1.729562,1.432968,2.325303
1,-1.586890,0.291956,1.311468


In [65]:
#2 . Remove rows with *all* missing values.
df.dropna(how="all")

,0,1,2
0,1.729562,1.432968,2.325303
1,-1.586890,0.291956,1.311468
2,-1.165746,NaN,-0.655313
3,NaN,-0.386667,-0.043223
5,-0.782933,NaN,NaN


In [67]:
#3. Fill missing values with forward fill
df.fillna(method="ffill")


/var/folders/vf/p6tql7n155x4cbnbq23vqytr0000gn/T/ipykernel_29753/3731247311.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")


,0,1,2
0,1.729562,1.432968,2.325303
1,-1.586890,0.291956,1.311468
2,-1.165746,0.291956,-0.655313
3,-1.165746,-0.386667,-0.043223
4,-1.165746,-0.386667,-0.043223
5,-0.782933,-0.386667,-0.043223


In [68]:
# 4. Fill missing values with mean of the column.
df.fillna(df.mean())

,0,1,2
0,1.729562,1.432968,2.325303
1,-1.586890,0.291956,1.311468
2,-1.165746,0.446085,-0.655313
3,-0.451502,-0.386667,-0.043223
4,-0.451502,0.446085,0.734559
5,-0.782933,0.446085,0.734559


## Part 2: Handling Duplicates

Duplicate rows can skew analysis and models. We typically identify them and remove them.

> **📊 Visual Illustration Available**: See ![duplicates](../visualisations/06_duplicates.png) for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


In [69]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`duplicated()` returns a boolean Series indicating whether each row has been seen before.

In [70]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates()` creates a new DataFrame with the duplicates removed.

In [71]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


**Subset:** Sometimes we only care about duplicates in specific columns.

In [ ]:
# range(7) creates a sequence of numbers from 0 to 6
# create a new column 'v1' with values 0 to 6
data["v1"] = range(7) 

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [ ]:
# Drop duplicates considering only column 'k1'
# keeps the first occurrence and drops subsequent duplicates
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


**Keep:** By default, it keeps the first occurrence. We can keep the last one instead.

In [75]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Part 3: Handling Outliers

Outliers are extreme values that deviate significantly from the rest of the data. We can filter them using boolean indexing.

> **📊 Visual Illustration Available**: See ![outliers](../visualisations/04_outliers.png) for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


In [130]:
# Create a dataset with normal distribution
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.060153,0.042585,-0.030650,0.008835
std,1.023344,0.997549,1.006908,1.007562
min,-3.136777,-3.197343,-3.946860,-3.570845
25%,-0.592801,-0.609170,-0.667530,-0.626969
50%,0.093357,0.037054,-0.045361,0.017333
75%,0.737922,0.696242,0.630762,0.697965
max,3.150054,3.398000,5.142250,3.062641


**Detection:** Let's find values exceeding 3 in absolute value (Standard Deviation > 3).

In [131]:
col = data[2]

# Boolean indexing to find rows where absolute value > 3
col[col.abs() > 3]

120   -3.946860
148    5.142250
161   -3.253418
304   -3.335430
Name: 2, dtype: float64

To find **any row** that has an outlier in **any column**, we use `.any(axis=1)`.

In [79]:
# 1. data.abs() > 3 returns a boolean DataFrame
# 2. .any(axis="columns") checks if any value in the row is True
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
54,-0.694332,-3.499437,-0.494365,1.149782
181,0.328210,-3.136782,1.663725,0.555813
256,-2.143357,0.695656,3.432855,-0.111732
292,-0.756013,0.311335,1.654634,3.204448
521,0.458169,-0.299577,-3.075963,-0.623349
626,0.210481,3.222230,0.062569,-1.248784
787,-1.629945,2.294429,0.172857,3.536920
895,0.720370,-3.230202,1.429198,-0.697165


**Capping:** Instead of removing outliers, we can cap them at a threshold.

In [145]:
# Set values > 3 to 3, and < -3 to -3, preserving the sign
# np.sign(): preserves positive/negative sign of the original value (+1, -1, or 0)
data[data.abs() > 3] = np.sign(data) * 3

In [144]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.060139,0.042347,-0.029257,0.010718
std,1.017576,0.984910,0.982939,0.991629
min,-2.982630,-2.962952,-2.824016,-2.951464
25%,-0.592801,-0.609170,-0.667530,-0.626969
50%,0.093357,0.037054,-0.045361,0.017333
75%,0.737922,0.696242,0.630762,0.697965
max,2.716990,2.841557,2.761514,2.830933


**Removal:** Or we can just drop the rows with outliers.

In [ ]:
# Keep rows where ALL columns are within the threshold ( < 3)
data[(data.abs() < 3).all(axis="columns")]

# When to use? When outliers are clearly errors. e.g age > 150

,0,1,2,3
0,-1.258934,-1.354581,-2.604376,0.293543
1,-0.424863,-2.626431,0.095064,0.149141
2,0.879742,0.775038,-0.455595,1.649535
3,-1.802029,0.714241,-0.367654,-0.813211
4,0.830053,2.057130,-0.214655,1.730793
...,...,...,...,...
995,0.702357,0.334159,0.369960,-2.373185
996,1.362790,0.427305,0.652380,0.707173
997,-0.671671,-0.453834,-0.597885,0.307498
998,0.210457,-2.690494,-0.940374,-0.101091


---

# Section 3: Refinement & Transformation

**Goal:** Now that the data is clean, we need to transform it into the right format for analysis.

We will cover:
1.  **Mapping & Replacing:** Changing values based on a dictionary logic.
2.  **String Manipulation:** Cleaning text data using the `.str` accessor.
3.  **Exporting:** Saving your hard work to a file.

## Part 1: Transforming Data (Mapping)

Sometimes we need to add new columns based on existing ones. `map()` is perfect for this.

In [83]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


**Scenario:** We want to add a column indicating the animal source of each food.

In [84]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

In [ ]:
# .map() looks up the value in the 'food' column in our dictionary
# create a new column 'animal'
# look at data["food"] and map each value to the corresponding animal
# map with meat_to_animal dictionary
data["animal"] = data["food"].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


You can also pass a function to `map()` for custom logic.

In [86]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Replacing Values:** `replace` is a specialized version of map, great for fixing sentinel values (like -999 for missing data).

In [87]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [88]:
# Replace -999 with NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [89]:
# Replace multiple values at once
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [90]:
# Replace with different values (-999 -> NaN, -1000 -> 0)
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [ ]:
# Using a dictionary for clarity
data.replace({-999: np.nan, -1000: 0})

# Using dictionary is safer and more readable

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Exercise: Transforming Data

Try replacing values in a DataFrame.

In [92]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

,0,1,2
0,0.571975,0.857631,1.166582
1,-0.697519,-0.735887,-1.220446
2,-0.760717,-999.000000,1.077604
3,-0.063132,-999.000000,-0.754642
4,0.160111,-999.000000,999.000000
5,0.758587,-999.000000,999.000000


> **Task:** Replace -999 with NaN and 999 with 0.

In [98]:
df.replace({-999: np.nan, 999: 0})


,0,1,2
0,0.571975,0.857631,1.166582
1,-0.697519,-0.735887,-1.220446
2,-0.760717,NaN,1.077604
3,-0.063132,NaN,-0.754642
4,0.160111,NaN,0.000000
5,0.758587,NaN,0.000000


## Part 2: String Manipulation

Pandas has a special accessor `.str` that unlocks string methods for an entire Series at once. This handles missing values gracefully.

In [99]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [100]:
# Check if 'gmail' exists in each string
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note on Data Types: Pandas has a specialized `StringDType` (`string`) vs the generic `object` type.

In [102]:
print(data.dtype)

object


In [101]:
data_as_string = data.astype('string')

data_as_string

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [103]:
data_as_string.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

**Slicing:** We can treat the column like a Python string.

In [104]:
# Get the first 5 characters
data_as_string.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes       <NA>
dtype: string

**Regex:** Regular expressions allow for complex pattern matching.

In [105]:
import re

In [106]:
# Pattern to identify email parts: (username) @ (domain) . (suffix)
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE) # ignore case sensitivity

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

**Retrieving elements:** We can chain `.str` calls to get specific parts of the regex match.

In [107]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [108]:
# Get index 1 of the tuple (the domain name)
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

The `extract` method is very powerful—it creates a new DataFrame with columns for each captured regex group.

In [109]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


### Exercise: Strings

> 1. Get the 1st group of the regex email pattern (the username).
> 2. Convert data into titlecase (e.g. "Dave@Google.Com").

In [113]:
#1. Get the 1st group of the regex email pattern (the username).
matches.str.get(0)

Dave      dave
Steve    steve
Rob        rob
Wes        NaN
dtype: object

In [114]:
#2. Convert data into titlecase (e.g. "Dave@Google.Com").
data.str.title()

Dave     Dave@Google.Com
Steve    Steve@Gmail.Com
Rob        Rob@Gmail.Com
Wes                  NaN
dtype: object

## Part 3: Writing Data (Exporting)

Once your data is cleaned, you need to save it. `to_csv` is the most common method.

In [ ]:
# export to CSV
result.to_csv("../data/out.csv") 

In [116]:
!cat ../data/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,


**Tip:** You usually want `index=False` to avoid saving the row numbers as a separate column.

In [117]:
result.to_csv("../data/out.csv", index=False)

In [118]:
!cat ../data/out.csv

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,


**Excel Export:**

In [ ]:
# Option 1: Using ExcelWriter (good for multiple sheets)
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()

In [120]:
# Option 2: Direct export
frame.to_excel('../data/out.xlsx')

---

## Optional Topics

If time permits, explore these advanced transformations.

### Renaming Axis Indexes
Changing row/column labels using mapping.

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

Using `.map()` on the index:

In [ ]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

In [ ]:
# Assign back to modify in-place
data.index = data.index.map(transform)

In [ ]:
data

Using `.rename()` (returns a copy by default):

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

### Permutation and Random Sampling
Reordering or selecting random subsets.

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

In [ ]:
# Create a random order of indices
sampler = np.random.permutation(5)

sampler

In [ ]:
# Reorder rows
df.iloc[sampler]

In [ ]:
# Alternative: use .take()
df.take(sampler)

Permuting columns:

In [ ]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

In [ ]:
df.take(column_sampler, axis=1)

**Random Sample:** Getting a random subset without manually creating a sampler.

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])

# Sample with replacement (items can be picked more than once)
choices.sample(n=10, replace=True)

> **Task:** Sample `df` using the parameter `frac` (percentage) instead of `n` (count).

### Categorical Data
Converting string columns to `category` dtype saves memory and speeds up operations.

> **📊 Visual Illustration Available**: See ![categorical_binning](../visualisations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


In [ ]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

In [ ]:
values.unique()

In [ ]:
values.value_counts()

**Under the hood:** Categoricals are stored as integers referencing a dictionary of values.

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

In [ ]:
dim

In [ ]:
dim.take(values)

**Using Pandas `category` type:**

In [ ]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

In [ ]:
# Convert object column to category
fruit_cat = df['fruit'].astype('category')

fruit_cat

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [ ]:
c = fruit_cat.array

type(c)

In [ ]:
c.categories

In [ ]:
c.codes

In [ ]:
dict(enumerate(c.categories))

In [ ]:
# Assign back to the dataframe
df['fruit'] = df['fruit'].astype('category')

df

**Creating from Codes:**

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

**Ordered Categoricals:** Useful for Likert scales or sizes (Small < Medium < Large).

In [ ]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

**Binning Data (`pd.cut`):** Converting continuous data (e.g., Age) into categorical bins (e.g., Age Groups).

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
# Define bin edges
bins = [18, 25, 35, 60, 100]

# Cut the data
age_cat = pd.cut(ages, bins)

age_cat

In [ ]:
age_cat.codes

In [ ]:
age_cat.categories

In [ ]:
age_cat.categories[0]

In [ ]:
pd.value_counts(age_cat)

**Binning Parameters:**

In [ ]:
# right=False makes the left side closed [inclusive, exclusive)
pd.cut(ages, bins, right=False)

In [ ]:
# Adding custom labels
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to `pd.cut`, it will compute equal-length bins based on the min/max of the data.

In [ ]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

> **Task:** Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

**Categorical Methods (.cat):**

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

Modifying categories:

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

In [ ]:
cat_s2.cat.remove_unused_categories()

**Computing Indicator / Dummy Variables:** Converting categorical variables into binary columns (One-Hot Encoding) for machine learning.

In [ ]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

In [ ]:
pd.get_dummies(df["key"])

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

Recipe: Combining `get_dummies` with `cut`.

In [ ]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

> **Task:** Cut values into 4 bins and create dummy variables.

### Databases

Connecting to a database using `sqlalchemy`.

In [ ]:
import sqlalchemy as sqla

In [ ]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [ ]:
# Create connection string
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

In [ ]:
# Read entire table
df = pd.read_sql('resale_flat_prices_2017', engine)

In [ ]:
df

In [ ]:
# Execute raw SQL query
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [ ]:
engine.table_names()

### Binary files (Pickle)

`pickle` is Python's native serialization format. Good for short-term storage, but not for sharing data between different languages.

In [ ]:
result.to_pickle('../data/out.pkl')

In [ ]:
data = pd.read_pickle('../data/out.pkl')

data

> **Task:** Write a filtered DataFrame to a new database table.
> 1. Filter for flats in "YISHUN".
> 2. Write to table `yishun_flat_prices_2017`.

In [ ]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

> **Final Challenge:** 
> 1. Read only flats from `BISHAN` to a new dataframe.
> 2. Write the dataframe to a new database table `bishan_flat_prices_2017`.